In [7]:
using JuMP
using Gurobi
using DataFrames

m = Model()

@variable(m, Sup[i = 1:10, j = 1:5],Bin)

df = readtable("/Users/lyynikki/Desktop/1.csv")
cost_raw = df[:Cost_NT_]
ul = df[:UL_Certified]
extra_cost = df[:Extra_Pre_Assembly_Cost_NT_]
defect_raw = df[:Defect_Rate]
total_cost_unshaped = [(cost_raw[i]+extra_cost[i])for i=1:43]

UL = zeros(10,5)
UL[1,1:4] = ul[1:4]
UL[2,1:5] = ul[5:9]
UL[3,1:4] = ul[10:13]
UL[4,1:5] = ul[14:18]
UL[5,1:4] = ul[19:22]
UL[6,1:4] = ul[23:26]
UL[7,1:4] = ul[27:30]
UL[8,1:5] = ul[31:35]
UL[9,1:5] = ul[36:40]
UL[10,1:3] = ul[41:43]
UL

Cost = zeros(10,5)
Cost[1,1:4] = total_cost_unshaped[1:4]
Cost[2,1:5] = total_cost_unshaped[5:9]
Cost[3,1:4] = total_cost_unshaped[10:13]
Cost[4,1:5] = total_cost_unshaped[14:18]
Cost[5,1:4] = total_cost_unshaped[19:22]
Cost[6,1:4] = total_cost_unshaped[23:26]
Cost[7,1:4] = total_cost_unshaped[27:30]
Cost[8,1:5] = total_cost_unshaped[31:35]
Cost[9,1:5] = total_cost_unshaped[36:40]
Cost[10,1:3] = total_cost_unshaped[41:43]
Cost

Defect = zeros(10,5)
Defect[1,1:4] = defect_raw[1:4]
Defect[2,1:5] = defect_raw[5:9]
Defect[3,1:4] = defect_raw[10:13]
Defect[4,1:5] = defect_raw[14:18]
Defect[5,1:4] = defect_raw[19:22]
Defect[6,1:4] = defect_raw[23:26]
Defect[7,1:4] = defect_raw[27:30]
Defect[8,1:5] = defect_raw[31:35]
Defect[9,1:5] = defect_raw[36:40]
Defect[10,1:3] = defect_raw[41:43]
Defect

Cost

10×5 Array{Float64,2}:
 1380.0  1668.0   1464.0  1620.0    0.0
  278.4   289.2    252.0   264.0  270.0
  934.0   799.0    883.0   899.6    0.0
  271.0   278.0    257.0   292.0  299.0
  213.0   221.25   229.5   218.0    0.0
  684.0   596.0    460.0   560.0    0.0
  679.0   727.0    628.0   631.0    0.0
  275.0   257.0    260.0   300.0  310.0
  420.0   404.0    388.0   388.0  412.0
  203.0   215.0    212.0     0.0    0.0

In [11]:
@objective(m, Min, sum{Defect[i,j]*Sup[i,j]*Cost[i,j], i=1:10, j=1:5})
for i=1:10
    @constraint(m, sum{Sup[i, j], j = 1:5} == 1)
end

@constraint(m, Sup[1,3]+Sup[2,3]<=1)
@constraint(m, sum{UL[i,j]*Sup[i,j],i = 1:10, j = 1:5} >= 7)
@constraint(m,Sup[2,1]== Sup[4,2])
@constraint(m,Sup[1,5] == Sup[3,5])
@constraint(m,Sup[5,5] == Sup[6,5])
@constraint(m,Sup[7,5] == Sup[10,5])
@constraint(m,Sup[10,4] == 0)
@constraint(m,Sup[1,5] == 0)
@constraint(m,Sup[5,5] == 0)
@constraint(m,Sup[7,5] == 0)

for i=1:10
    @constraint(m,sum{(1+Defect[i,j])*Sup[i,j]*Cost[i,j],i=1:10, j=1:5}<= 5400*33.43)
end

status = solve(m)
println("Objective value is ", getobjectivevalue(m) )
getvalue(Sup)

Optimize a model with 30 rows, 50 columns and 525 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+03]
  Objective range [3e+00, 8e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 2e+05]
Found heuristic solution: objective 227.331
Presolve removed 30 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 9.148060000000e+01, best bound 9.148060000000e+01, gap 0.0%
Objective value is 91.48060000000001


10×5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0

In [14]:
total_cost_unshaped = [(cost_raw[i]+extra_cost[i]) *(1+defect_raw[i]) for i=1:43]
Cost = zeros(10,5)
Cost[1,1:4] = total_cost_unshaped[1:4]
Cost[2,1:5] = total_cost_unshaped[5:9]
Cost[3,1:4] = total_cost_unshaped[10:13]
Cost[4,1:5] = total_cost_unshaped[14:18]
Cost[5,1:4] = total_cost_unshaped[19:22]
Cost[6,1:4] = total_cost_unshaped[23:26]
Cost[7,1:4] = total_cost_unshaped[27:30]
Cost[8,1:5] = total_cost_unshaped[31:35]
Cost[9,1:5] = total_cost_unshaped[36:40]
Cost[10,1:3] = total_cost_unshaped[41:43]
Cost

x = getvalue(Sup)
Cost.*x
sum(Cost.*x)

10×5 Array{Float64,2}:
   0.0      0.0    1485.96   0.0    0.0  
   0.0    292.96      0.0    0.0    0.0  
   0.0      0.0     892.713  0.0    0.0  
   0.0      0.0       0.0    0.0  303.784
 227.484    0.0       0.0    0.0    0.0  
   0.0    604.344     0.0    0.0    0.0  
   0.0    736.451     0.0    0.0    0.0  
   0.0      0.0       0.0    0.0  314.34 
 427.98     0.0       0.0    0.0    0.0  
   0.0    221.665     0.0    0.0    0.0  

In [16]:
y = ones(10,5)
x.*(y .- Defect)

10×5 Array{Float64,2}:
 0.0    0.0    0.985  0.0  0.0  
 0.0    0.987  0.0    0.0  0.0  
 0.0    0.0    0.989  0.0  0.0  
 0.0    0.0    0.0    0.0  0.984
 0.932  0.0    0.0    0.0  0.0  
 0.0    0.986  0.0    0.0  0.0  
 0.0    0.987  0.0    0.0  0.0  
 0.0    0.0    0.0    0.0  0.986
 0.981  0.0    0.0    0.0  0.0  
 0.0    0.969  0.0    0.0  0.0  

In [23]:
y = ones(10,5)
x.*(y .- Defect)



10×5 Array{Float64,2}:
 0.0        0.0        0.0122482  0.0  0.0      
 0.0        0.0105936  0.0        0.0  0.0      
 0.0        0.0        0.0089457  0.0  0.0      
 0.0        0.0        0.0        0.0  0.013078 
 0.0586828  0.0        0.0        0.0  0.0      
 0.0        0.0114201  0.0        0.0  0.0      
 0.0        0.0105936  0.0        0.0  0.0      
 0.0        0.0        0.0        0.0  0.0114201
 0.0155777  0.0        0.0        0.0  0.0      
 0.0        0.025731   0.0        0.0  0.0      

In [25]:
a = (Defect ./(y .- Defect)).*x*0.8043
sum(a) + 0.8043

0.9825908393864567